In [12]:
import os, sys
from dotenv import load_dotenv
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [2]:
try:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("Gemini api key set successfully.")
except Exception as e:
    print("Error setting Gemini api key:", e)

Gemini api key set successfully.


In [16]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.mcp_tool import StreamableHTTPConnectionParams
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [19]:
# MCP integration with Everything Server
mcp_image_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",  # Run MCP server via npx
            args=[
                "-y",  # Argument for npx to auto-confirm install
                "@modelcontextprotocol/server-everything",
            ],
            tool_filter=["getTinyImage"],
        ),
        timeout=30,
    )
)

print("✅ MCP Tool created")

✅ MCP Tool created


In [20]:
image_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="image_agent",
    instruction="Use the MCP Tool to generate images for user queries",
    tools=[mcp_image_server],
)

In [21]:
from google.adk.runners import InMemoryRunner

runner = InMemoryRunner(agent=image_agent)

In [24]:
response = ""

In [25]:
response = await runner.run_debug("Provide a sample tiny image", verbose=True)


 ### Continue session: debug_session_id

User > Provide a sample tiny image


UnsupportedOperation: fileno

In [26]:
from IPython.display import display, Image as IPImage
import base64

for event in response:
    if event.content and event.content.parts:
        for part in event.content.parts:
            if hasattr(part, "function_response") and part.function_response:
                for item in part.function_response.response.get("content", []):
                    if item.get("type") == "image":
                        display(IPImage(data=base64.b64decode(item["data"])))

In [27]:
LARGE_ORDER_THRESHOLD = 5


def place_shipping_order(
    num_containers: int, destination: str, tool_context: ToolContext
) -> dict:
    """Places a shipping order. Requires approval if ordering more than 5 containers (LARGE_ORDER_THRESHOLD).

    Args:
        num_containers: Number of containers to ship
        destination: Shipping destination

    Returns:
        Dictionary with order status
    """

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # SCENARIO 1: Small orders (≤5 containers) auto-approve
    if num_containers <= LARGE_ORDER_THRESHOLD:
        return {
            "status": "approved",
            "order_id": f"ORD-{num_containers}-AUTO",
            "num_containers": num_containers,
            "destination": destination,
            "message": f"Order auto-approved: {num_containers} containers to {destination}",
        }

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # SCENARIO 2: This is the first time this tool is called. Large orders need human approval - PAUSE here.
    if not tool_context.tool_confirmation:
        tool_context.request_confirmation(
            hint=f"⚠️ Large order: {num_containers} containers to {destination}. Do you want to approve?",
            payload={"num_containers": num_containers, "destination": destination},
        )
        return {  # This is sent to the Agent
            "status": "pending",
            "message": f"Order for {num_containers} containers requires approval",
        }

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # SCENARIO 3: The tool is called AGAIN and is now resuming. Handle approval response - RESUME here.
    if tool_context.tool_confirmation.confirmed:
        return {
            "status": "approved",
            "order_id": f"ORD-{num_containers}-HUMAN",
            "num_containers": num_containers,
            "destination": destination,
            "message": f"Order approved: {num_containers} containers to {destination}",
        }
    else:
        return {
            "status": "rejected",
            "message": f"Order rejected: {num_containers} containers to {destination}",
        }


print("✅ Long-running functions created!")

✅ Long-running functions created!


In [28]:
# Create shipping agent with pausable tool
shipping_agent = LlmAgent(
    name="shipping_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a shipping coordinator assistant.
  
  When users request to ship containers:
   1. Use the place_shipping_order tool with the number of containers and destination
   2. If the order status is 'pending', inform the user that approval is required
   3. After receiving the final result, provide a clear summary including:
      - Order status (approved/rejected)
      - Order ID (if available)
      - Number of containers and destination
   4. Keep responses concise but informative
  """,
    tools=[FunctionTool(func=place_shipping_order)],
)

print("✅ Shipping Agent created!")

✅ Shipping Agent created!


In [29]:
# Wrap the agent in a resumable app - THIS IS THE KEY FOR LONG-RUNNING OPERATIONS!
shipping_app = App(
    name="shipping_coordinator",
    root_agent=shipping_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)

print("✅ Resumable app created!")

✅ Resumable app created!


C:\Users\Manan\AppData\Local\Temp\ipykernel_39276\3673777575.py:5: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),


In [31]:
session_service = InMemorySessionService()

# Create runner with the resumable app
shipping_runner = Runner(
    app=shipping_app,  # Pass the app instead of the agent
    session_service=session_service,
)

print("✅ Runner created!")

✅ Runner created!


In [32]:
def check_for_approval(events):
    """Check if events contain an approval request.

    Returns:
        dict with approval details or None
    """
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if (
                    part.function_call
                    and part.function_call.name == "adk_request_confirmation"
                ):
                    return {
                        "approval_id": part.function_call.id,
                        "invocation_id": event.invocation_id,
                    }
    return None

In [33]:
def print_agent_response(events):
    """Print agent's text responses from events."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"Agent > {part.text}")

In [34]:
def create_approval_response(approval_info, approved):
    """Create approval response message."""
    confirmation_response = types.FunctionResponse(
        id=approval_info["approval_id"],
        name="adk_request_confirmation",
        response={"confirmed": approved},
    )
    return types.Content(
        role="user", parts=[types.Part(function_response=confirmation_response)]
    )


print("✅ Helper functions defined")

✅ Helper functions defined


In [35]:
async def run_shipping_workflow(query: str, auto_approve: bool = True):
    """Runs a shipping workflow with approval handling.

    Args:
        query: User's shipping request
        auto_approve: Whether to auto-approve large orders (simulates human decision)
    """

    print(f"\n{'='*60}")
    print(f"User > {query}\n")

    # Generate unique session ID
    session_id = f"order_{uuid.uuid4().hex[:8]}"

    # Create session
    await session_service.create_session(
        app_name="shipping_coordinator", user_id="test_user", session_id=session_id
    )

    query_content = types.Content(role="user", parts=[types.Part(text=query)])
    events = []

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 1: Send initial request to the Agent. If num_containers > 5, the Agent returns the special `adk_request_confirmation` event
    async for event in shipping_runner.run_async(
        user_id="test_user", session_id=session_id, new_message=query_content
    ):
        events.append(event)

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 2: Loop through all the events generated and check if `adk_request_confirmation` is present.
    approval_info = check_for_approval(events)

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 3: If the event is present, it's a large order - HANDLE APPROVAL WORKFLOW
    if approval_info:
        print(f"⏸️  Pausing for approval...")
        print(f"🤔 Human Decision: {'APPROVE ✅' if auto_approve else 'REJECT ❌'}\n")

        # PATH A: Resume the agent by calling run_async() again with the approval decision
        async for event in shipping_runner.run_async(
            user_id="test_user",
            session_id=session_id,
            new_message=create_approval_response(
                approval_info, auto_approve
            ),  # Send human decision here
            invocation_id=approval_info[
                "invocation_id"
            ],  # Critical: same invocation_id tells ADK to RESUME
        ):
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if part.text:
                        print(f"Agent > {part.text}")

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    else:
        # PATH B: If the `adk_request_confirmation` is not present - no approval needed - order completed immediately.
        print_agent_response(events)

    print(f"{'='*60}\n")


print("✅ Workflow function ready")

✅ Workflow function ready


In [36]:
# Demo 1: It's a small order. Agent receives auto-approved status from tool
await run_shipping_workflow("Ship 3 containers to Singapore")

# Demo 2: Workflow simulates human decision: APPROVE ✅
await run_shipping_workflow("Ship 10 containers to Rotterdam", auto_approve=True)

# Demo 3: Workflow simulates human decision: REJECT ❌
await run_shipping_workflow("Ship 8 containers to Los Angeles", auto_approve=False)


User > Ship 3 containers to Singapore



Agent > Shipping order approved! 3 containers will be shipped to Singapore. The order ID is ORD-3-AUTO.


User > Ship 10 containers to Rotterdam



c:\Users\Manan\anaconda3\envs\adk\Lib\site-packages\google\adk\tools\tool_context.py:92: UserWarning: [EXPERIMENTAL] ToolConfirmation: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  ToolConfirmation(
c:\Users\Manan\anaconda3\envs\adk\Lib\site-packages\google\adk\agents\invocation_context.py:298: UserWarning: [EXPERIMENTAL] BaseAgentState: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  self.agent_states[event.author] = BaseAgentState()


⏸️  Pausing for approval...
🤔 Human Decision: APPROVE ✅



User > Ship 8 containers to Los Angeles



⏸️  Pausing for approval...
🤔 Human Decision: REJECT ❌

Agent > Shipping order for 8 containers to Los Angeles has been rejected.



In [40]:
import base64, textwrap, uuid
from typing import List, Dict

In [37]:
# 1x1 transparent PNG base64 (placeholder "image")
TINY_PNG_B64 = (
    "iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVR4nGMAAQAABQAB"
    "p2ZqSgAAAABJRU5ErkJggg=="
)

In [42]:
def fake_generate_image(prompt: str) -> Dict:
    """
    Simulate generating a single image for a prompt. Returns base64 and a fake id.
    """
    return {
        "status": "success",
        "image_id": str(uuid.uuid4()),
        "prompt": prompt,
        "image_b64": TINY_PNG_B64,
        "note": "FAKE image for local practice",
    }

In [43]:
BULK_THRESHOLD = 1   # >1 requires approval
USE_MCP = False # Switch between MCP and fake tool

In [44]:
def request_images(prompt: str, n: int) -> dict:
    """
    Approval gate.
    - If n <= BULK_THRESHOLD: auto-approve -> {"status":"approved", ...}
    - If n >  BULK_THRESHOLD: ask user to approve -> {"status":"needs_approval", ...}
    """
    if n <= BULK_THRESHOLD:
        return {
            "status": "approved",
            "prompt": prompt,
            "n": n,
            "message": f"Auto-approved single image request (n={n})."
        }
    else:
        est_cost = n * 0.02  # pretend cost: $0.02 / image (for practice)
        return {
            "status": "needs_approval",
            "prompt": prompt,
            "n": n,
            "estimate_usd": round(est_cost, 2),
            "message": (
                f"Bulk request detected: {n} images. Estimated cost ≈ ${est_cost:.2f}. "
                "Reply 'approve' to proceed, or 'cancel' to abort."
            )
        }

In [45]:
def generate_images(prompt: str, n: int) -> dict:
    """
    Fake image generator for local practice.
    Returns a list of {image_id, image_b64}.
    """
    images = [fake_generate_image(prompt) for _ in range(max(1, n))]
    return {"status": "success", "count": len(images), "images": images}

In [46]:
image_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite"),
    name="image_agent",
    instruction=textwrap.dedent(f"""
        You are an image-generation orchestrator with a cost guard.

        RULES:
        - When the user asks for images, ALWAYS call the tool request_images(prompt, n) first.
          * If the user didn't give a number, ask for it; default to 1 if they say "one".
        - If request_images returns status=approved -> immediately call generate_images(prompt, n).
        - If request_images returns status=needs_approval:
            1) Clearly show n and the estimated cost,
            2) Ask the user to confirm with 'approve' or 'cancel'.
            3) If the user approves, call generate_images(prompt, n). If they cancel, end politely.

        OUTPUT:
        - After generation, return a concise summary and the image IDs.
        - Do not reveal base64 by default; only show IDs. You may say that images are available in memory.
    """).strip(),
    tools=[request_images, generate_images],   # (MCP toolset can be added later)
)

In [47]:
runner = InMemoryRunner(agent=image_agent)

In [48]:
resp = await runner.run_debug(
    "Generate 1 image of a red cube in isometric pixel art.",
    verbose=True
)
resp


 ### Created new session: debug_session_id

User > Generate 1 image of a red cube in isometric pixel art.


image_agent > [Calling tool: request_images({'n': 1, 'prompt': 'A red cube in isometric pixel ...)]
image_agent > [Tool result: {'status': 'approved', 'prompt': 'A red cube in isometric pixel art.', 'n': 1, 'message': 'Auto-appr...]


image_agent > [Calling tool: generate_images({'n': 1, 'prompt': 'A red cube in isometric pixel ...)]
image_agent > [Tool result: {'status': 'success', 'count': 1, 'images': [{'status': 'success', 'image_id': '2e7c70af-8339-499c-8...]
image_agent > Here are the images you requested:

2e7c70af-8339-499c-855f-b2e532bbd60d


[Event(model_version='gemini-2.5-flash-lite', content=Content(
   parts=[
     Part(
       function_call=FunctionCall(
         args={
           'n': 1,
           'prompt': 'A red cube in isometric pixel art.'
         },
         id='adk-b1db0beb-0b8b-406b-81cc-b310950c225f',
         name='request_images'
       )
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=26,
   prompt_token_count=395,
   prompt_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=395
     ),
   ],
   total_token_count=421
 ), live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=None, logprobs_result=None, cache_metadata=None, citation_metadata=None, invocation_id='e

In [49]:
resp = await runner.run_debug(
    "Generate 5 tiny images of mountains at sunrise.",
    verbose=True
)
resp


 ### Continue session: debug_session_id

User > Generate 5 tiny images of mountains at sunrise.


image_agent > [Calling tool: request_images({'prompt': '5 tiny images of mountains at sunrise....)]
image_agent > [Tool result: {'status': 'needs_approval', 'prompt': '5 tiny images of mountains at sunrise.', 'n': 5, 'estimate_u...]
image_agent > The estimated cost is $0.10 for 5 images. Please approve or cancel.


[Event(model_version='gemini-2.5-flash-lite', content=Content(
   parts=[
     Part(
       function_call=FunctionCall(
         args={
           'n': 5,
           'prompt': '5 tiny images of mountains at sunrise.'
         },
         id='adk-7f7e7d9f-4fab-4e3d-8a37-4657a7fff34d',
         name='request_images'
       )
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=26,
   prompt_token_count=706,
   prompt_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=706
     ),
   ],
   total_token_count=732
 ), live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=None, logprobs_result=None, cache_metadata=None, citation_metadata=None, invocation_i

In [50]:
resp2 = await runner.run_debug("approve", verbose=True)
resp2


 ### Continue session: debug_session_id

User > approve


image_agent > [Calling tool: generate_images({'prompt': '5 tiny images of mountains at sunrise....)]
image_agent > [Tool result: {'status': 'success', 'count': 5, 'images': [{'status': 'success', 'image_id': '04227670-8b9f-4c4a-a...]
image_agent > Here are the images you requested:

04227670-8b9f-4c4a-a22a-565bff7263e3
84784f5c-a8ee-4420-b65b-c297fc6b43ea
932e4769-c9af-409e-86bd-eebabbc07eba
bb3f0433-d2a9-4a7c-b426-515d5b045715
5a4bbbee-2c67-40d3-a55a-80428bc8143a


[Event(model_version='gemini-2.5-flash-lite', content=Content(
   parts=[
     Part(
       function_call=FunctionCall(
         args={
           'n': 5,
           'prompt': '5 tiny images of mountains at sunrise.'
         },
         id='adk-6ecf11d7-0664-476f-a023-ab14ff889ad9',
         name='generate_images'
       )
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=26,
   prompt_token_count=834,
   prompt_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=834
     ),
   ],
   total_token_count=860
 ), live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=None, logprobs_result=None, cache_metadata=None, citation_metadata=None, invocation_